# DELL Eagle Model

In [1]:
import torch
from torch import nn
import math
import matplotlib.pyplot as plt

In [2]:

from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


## Load Data

In [3]:
import numpy as np

real = np.load('/content/drive/MyDrive/Footprints Datasets/real.npz')
fake = np.load('/content/drive/MyDrive/Footprints Datasets/fake.npz')

In [ ]:
fake

In [ ]:
real.files
real['y'].shape


In [ ]:
real['x'].shape

In [ ]:
fake['x'].shape

In [4]:
print(real['x'][2][728])

[2.3680e+04 3.2576e+04        inf 5.0848e+04 4.3280e+03 1.6288e+02
 1.0450e+02 8.6188e+01 1.9094e+01 1.2742e+01 1.5025e+02 9.2400e+02
 1.4210e+03 4.8675e+02 1.8156e+01 1.2050e+02 3.2650e+02 5.8550e+02
 5.1100e+02 1.3300e+02 5.0812e+01 5.6156e+01 2.0662e+02 4.0025e+02
 5.9500e+02 9.9300e+02 9.2450e+02 4.4375e+02 6.4625e+01 1.8016e+01
 3.2844e+01 6.0281e+01 5.4188e+01 6.5062e+01 1.4047e+01 6.5391e+00
 4.2688e+01 7.7000e+01 1.9594e+01 1.9873e-01 2.4648e+00 1.8496e+00
 3.0312e+00 7.0264e-01 1.3574e+00 1.4180e+00 9.9170e-01 4.6143e-01
 1.8398e+00 3.6367e+00 4.3384e-01 8.7939e-01 2.3652e+00 1.1475e+00
 5.1562e+00 1.6904e+00 1.0068e+00 7.1973e-01 2.0972e-01 1.1563e-05
 1.9019e-01 2.1692e-01 2.2949e-01 7.1472e-02 6.5491e-02 3.0005e-01
 2.0728e-01 1.1279e-01 4.5319e-02 3.1860e-01 3.4375e-01 2.8735e-01
 3.8794e-01 5.9277e-01 2.9388e-02 1.4001e-01 1.3391e-01 9.0918e-01
 7.6025e-01 1.1749e-01 2.9590e-01 7.9346e-01 2.9590e-01 8.5449e-02
 1.6699e-01 6.2256e-02 1.4107e-02 4.9164e-02 4.9377e-02 2.3682

## Data Preprocessing

### Handle inf values

In [6]:
# prompt: train the model on real['x'] and fake['x] with the labels being 1 for real and 0 for fake

import numpy as np
# Prepare the training data
value = 65470.0

real_x = real['x']
fake_x = fake['x']

real_x = np.nan_to_num(real_x, value)
fake_x = np.nan_to_num(fake_x, value)


# Prepare the training labels
real_y = np.ones((real_x.shape[0], 1))
fake_y = np.zeros((fake_x.shape[0], 1))




### Standardize Data

In [7]:
# prompt: standardize the datasets

import numpy as np
# Standardize the real and fake datasets
mean_real = np.mean(real_x, axis=0)
std_real = np.std(real_x, axis=0)
real_x = (real_x - mean_real) / std_real

mean_fake = np.mean(fake_x, axis=0)
std_fake = np.std(fake_x, axis=0)
fake_x = (fake_x - mean_fake) / std_fake


/usr/local/lib/python3.10/dist-packages/numpy/core/_methods.py:152: RuntimeWarning: overflow encountered in reduce
  arrmean = umr_sum(arr, axis, dtype, keepdims=True, where=where)
/usr/local/lib/python3.10/dist-packages/numpy/core/_methods.py:176: RuntimeWarning: overflow encountered in multiply
  x = um.multiply(x, x, out=x)
/usr/local/lib/python3.10/dist-packages/numpy/core/_methods.py:187: RuntimeWarning: overflow encountered in reduce
  ret = umr_sum(x, axis, dtype, out, keepdims=keepdims, where=where)
<ipython-input-7-65ac2c54ea12>:7: RuntimeWarning: divide by zero encountered in divide
  real_x = (real_x - mean_real) / std_real
<ipython-input-7-65ac2c54ea12>:7: RuntimeWarning: invalid value encountered in divide
  real_x = (real_x - mean_real) / std_real
<ipython-input-7-65ac2c54ea12>:11: RuntimeWarning: divide by zero encountered in divide
  fake_x = (fake_x - mean_fake) / std_fake
<ipython-input-7-65ac2c54ea12>:11: RuntimeWarning: invalid value encountered in divide
  fake_x =

In [8]:
import numpy as np
# Prepare the training data
value = 0.0


real_x = np.nan_to_num(real_x, value)
fake_x = np.nan_to_num(fake_x, value)


# Prepare the training labels
real_y = np.ones((real_x.shape[0], 1))
fake_y = np.zeros((fake_x.shape[0], 1))

### Split Data

In [9]:
# prompt: split the dataset using real_x, fake_x, real_y, fake_y

import numpy as np
# Split the dataset into training and test sets
from sklearn.model_selection import train_test_split

real_x_train, real_x_test, real_y_train, real_y_test = train_test_split(real_x, real_y, test_size=0.2)
fake_x_train, fake_x_test, fake_y_train, fake_y_test = train_test_split(fake_x, fake_y, test_size=0.2)

# Combine the training data
x_train = np.concatenate((real_x_train, fake_x_train))
y_train = np.concatenate((real_y_train, fake_y_train))

# Combine the test data
x_test = np.concatenate((real_x_test, fake_x_test))
y_test = np.concatenate((real_y_test, fake_y_test))


## XGBoost

### Flatten Data for XGBoost

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score, f1_score
from sklearn.ensemble import GradientBoostingClassifier

# Convert the data to a flat 2D shape
X_train_flat = x_train.reshape(x_train.shape[0], -1)
X_test_flat = x_test.reshape(x_test.shape[0], -1)

# Create Gradient Boosting classifier
model = GradientBoostingClassifier(random_state=42, verbose=1)
# Train the model
model.fit(X_train_flat, y_train)



/usr/local/lib/python3.10/dist-packages/sklearn/ensemble/_gb.py:437: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


      Iter       Train Loss   Remaining Time 
         1           1.1963           22.45m
         2           1.0407           25.79m
         3           0.9109           39.09m
         4           0.8012           35.16m
         5           0.7075           34.22m
         6           0.6268           34.58m
         7           0.5568           32.74m
         8           0.4957           30.23m
         9           0.4422           29.31m
        10           0.3951           28.56m
        20           0.1355           22.53m
        30           0.0487           16.87m
        40           0.0178           13.23m
        50           0.0065           10.41m
        60           0.0024            8.00m
        70           0.0009            5.82m
        80           0.0003            3.79m
        90           0.0001            1.86m
       100           0.0000            0.00s


GradientBoostingClassifier(random_state=42, verbose=1)

### Test Data

In [ ]:
# Make predictions
y_pred = model.predict(X_test_flat)

# Calculate accuracy and F1 score
accuracy = accuracy_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred)

print(f"Accuracy: {accuracy}")
print(f"F1 score: {f1}")


Accuracy: 0.9966666666666667
F1 score: 0.9966777408637874


### Save Model

In [ ]:
# Save Model Using Pickle
import pandas
from sklearn import model_selection
from sklearn.linear_model import LogisticRegression
import pickle

# save the model to disk
filename = 'finalized_model.sav'
pickle.dump(model, open(filename, 'wb'))

# some time later...

# load the model from disk
loaded_model = pickle.load(open(filename, 'rb'))
result = loaded_model.score(X_test_flat, y_test)
print(result)

0.9966666666666667


## CNN Models

In [ ]:
transform = transforms.Compose(
    [transforms.ToTensor(),
     transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

batch_size = 4

NameError: name 'transforms' is not defined

In [ ]:
# prompt: take the real['y'] and return for every entry a tuple consisting the index of the first and last occurence of one and store in a new numpy array called modifiied_y

import numpy as np
modified_y = np.zeros((real['y'].shape[0], 2))
for i in range(real['y'].shape[0]):
  index_first = np.where(real['y'][i] == 1)[0][0]
  index_last = np.where(real['y'][i] == 1)[0][-1]
  modified_y[i] = [index_first, index_last]

print(modified_y[0])


[122. 220.]


In [ ]:
fake_sample = fake['y'][0]

print(np.unique(fake['y']))

[0]


In [ ]:
transform = transforms.Compose(
    [transforms.ToTensor(),
     transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])


In [ ]:
# import torch.nn as nn
# import torch.nn.functional as F


# class Net(nn.Module):
#     def __init__(self):
#         super().__init__()
#         self.conv1 = nn.Conv2d(16, 6, 5)
#         self.pool = nn.MaxPool2d(2, 2)
#         self.conv2 = nn.Conv2d(6, 16, 5)
#         self.fc1 = nn.Linear(10912 , 120)
#         self.fc2 = nn.Linear(120, 84)
#         self.fc3 = nn.Linear(84, 10)

#     def forward(self, x):
#         x = self.pool(F.relu(self.conv1(x)))
#         x = self.pool(F.relu(self.conv2(x)))
#         x = torch.flatten(x, 1) # flatten all dimensions except batch
#         x = F.relu(self.fc1(x))
#         x = F.relu(self.fc2(x))
#         x = self.fc3(x)
#         return x


# net = Net()


# prompt: write a CNN model with 1 CNN layer, 1 max pooling, 1 CNN, 1 dense layer and an output of 496 nodes using tensorflow keras

import tensorflow as tf
from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, Flatten, Dense
from tensorflow.keras.models import Model

# Define the input layer
inputs = Input(shape=(real['x'].shape[1], real['x'].shape[2], 1))

# Add the first CNN layer
conv1 = Conv2D(filters=16, kernel_size=3, activation='relu')(inputs)

# Add the max pooling layer
pool1 = MaxPooling2D(pool_size=2)(conv1)

# Add the second CNN layer
conv2 = Conv2D(filters=32, kernel_size=4, activation='relu')(pool1)

# Add the max pooling layer
pool2 = MaxPooling2D(pool_size=3)(conv2)

# Add the second CNN layer
conv3 = Conv2D(filters=64, kernel_size=4, activation='relu')(pool2)

# Add the max pooling layer
pool3 = MaxPooling2D(pool_size=3)(conv3)

# Flatten the output of the CNN layers
flattened = Flatten()(pool2)


dense1 = Dense(128, activation='relu')(flattened)

# Add the dense layer
dense2 = Dense(1, activation='sigmoid')(dense1)

# Create the model
model = Model(inputs=inputs, outputs=dense2)

# Compile the model
model.compile(loss=tf.keras.losses.BinaryCrossentropy(), optimizer='SGD', metrics=['accuracy'])

# Print the model summary
model.summary()


Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 1998, 101, 1)]    0         
                                                                 
 conv2d (Conv2D)             (None, 1996, 99, 16)      160       
                                                                 
 max_pooling2d (MaxPooling2  (None, 998, 49, 16)       0         
 D)                                                              
                                                                 
 conv2d_1 (Conv2D)           (None, 995, 46, 32)       8224      
                                                                 
 max_pooling2d_1 (MaxPoolin  (None, 331, 15, 32)       0         
 g2D)                                                            
                                                                 
 flatten (Flatten)           (None, 158880)            0     

In [ ]:
# Combine the real and fake data
x_train = np.concatenate((real_x, fake_x), axis=0)
y_train = np.concatenate((real_y, fake_y), axis=0)

# Train the model
model.fit(x_train, y_train, epochs=10, batch_size=32)

Epoch 1/10
 3/47 [>.............................] - ETA: 3:54 - loss: nan - accuracy: 0.5104                  

KeyboardInterrupt: 

In [ ]:
import torch
import torchvision
import torchvision.transforms as transforms

In [ ]:
# prompt: use torch.Dataloader on x_train, x_test, y_train, y_test

train_dataset = torch.utils.data.TensorDataset(torch.from_numpy(x_train), torch.from_numpy(y_train))
test_dataset = torch.utils.data.TensorDataset(torch.from_numpy(x_test), torch.from_numpy(y_test))

train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=16, shuffle=True)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=16, shuffle=False)


In [ ]:
transform = transforms.Compose(
    [transforms.ToTensor(),
     transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

batch_size = 4

trainset =
trainloader = torch.utils.data.DataLoader(trainset, batch_size=batch_size,
                                          shuffle=True, num_workers=2)

testset = torchvision.datasets.CIFAR10(root='./data', train=False,
                                       download=True, transform=transform)
testloader = torch.utils.data.DataLoader(testset, batch_size=batch_size,
                                         shuffle=False, num_workers=2)

classes = ('plane', 'car', 'bird', 'cat',
           'deer', 'dog', 'frog', 'horse', 'ship', 'truck')

In [ ]:
import torch.nn as nn
import torch.nn.functional as F


class Net(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(16, 6, 5)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(6, 16, 5)
        self.fc1 = nn.Linear(10912 , 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = torch.flatten(x, 1) # flatten all dimensions except batch
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x


net = Net()

In [ ]:
import torch.optim as optim

criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(net.parameters(), lr=0.001, momentum=0.9)

In [ ]:
for epoch in range(2):  # loop over the dataset multiple times

    running_loss = 0.0
    for i, data in enumerate(train_loader, 0):
        # get the inputs; data is a list of [inputs, labels]
        inputs, labels = data
        # zero the parameter gradients
        optimizer.zero_grad()
        input_tensor = inputs.to(dtype=torch.float)
        labels = labels.squeeze().long()

        # forward + backward + optimize
        outputs = net(input_tensor)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        # print statistics
        running_loss += loss.item()
        if i % 2000 == 1999:    # print every 2000 mini-batches
            print(f'[{epoch + 1}, {i + 1:5d}] loss: {running_loss / 2000:.3f}')
            running_loss = 0.0

print('Finished Training')

RuntimeError: Given groups=1, weight of size [6, 16, 5, 5], expected input[1, 12, 1998, 101] to have 16 channels, but got 12 channels instead

In [ ]:
np.max(real['x'])

inf

In [ ]:
arr = np.array(real['x'])
indices = np.where(np.isinf(arr))
print("Infinite values:", indices)


Infinite values: (array([  2,   2,   2, ..., 749, 749, 749]), array([728, 732, 732, ..., 423, 423, 428]), array([2, 1, 2, ..., 2, 3, 3]))


In [ ]:
arr[2][728]

array([2.3680e+04, 3.2576e+04,        inf, 5.0848e+04, 4.3280e+03,
       1.6288e+02, 1.0450e+02, 8.6188e+01, 1.9094e+01, 1.2742e+01,
       1.5025e+02, 9.2400e+02, 1.4210e+03, 4.8675e+02, 1.8156e+01,
       1.2050e+02, 3.2650e+02, 5.8550e+02, 5.1100e+02, 1.3300e+02,
       5.0812e+01, 5.6156e+01, 2.0662e+02, 4.0025e+02, 5.9500e+02,
       9.9300e+02, 9.2450e+02, 4.4375e+02, 6.4625e+01, 1.8016e+01,
       3.2844e+01, 6.0281e+01, 5.4188e+01, 6.5062e+01, 1.4047e+01,
       6.5391e+00, 4.2688e+01, 7.7000e+01, 1.9594e+01, 1.9873e-01,
       2.4648e+00, 1.8496e+00, 3.0312e+00, 7.0264e-01, 1.3574e+00,
       1.4180e+00, 9.9170e-01, 4.6143e-01, 1.8398e+00, 3.6367e+00,
       4.3384e-01, 8.7939e-01, 2.3652e+00, 1.1475e+00, 5.1562e+00,
       1.6904e+00, 1.0068e+00, 7.1973e-01, 2.0972e-01, 1.1563e-05,
       1.9019e-01, 2.1692e-01, 2.2949e-01, 7.1472e-02, 6.5491e-02,
       3.0005e-01, 2.0728e-01, 1.1279e-01, 4.5319e-02, 3.1860e-01,
       3.4375e-01, 2.8735e-01, 3.8794e-01, 5.9277e-01, 2.9388e

In [ ]:
value = 65470.0

x_train[np.isinf(x_train)] = np.nan  # Replace infinite with NaN
x_train = np.nan_to_num(x_train, value)


In [ ]:
import numpy as np

# Create a numpy array
arr2 = np.array(real['x'])

# Find the maximum value
max_value = np.max(arr2)

# Replace the maximum value with negative infinity
arr2[arr2 == max_value] = -np.inf

# Find the second maximum value
second_max_value = np.max(arr2)

# print("Original array:", arr2)
print("Second maximum value:", second_max_value)


Second maximum value: 65470.0
